In [ ]:
# %matplotlib inline
import pandas  as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import math
import os;
from health.utils import  load_training_data ,slugify

%reload_ext autoreload
%autoreload 2


TARGET='Depression'

df = load_training_data('v1')
print("total rows",len(df)) 
# print(df['Sleep Duration'].unique())

column_names = df.keys().to_list()
column_names.remove('id')
 


In [ ]:

# Print the count of each distinct value in each column



for column in column_names:
    
    slug = slugify(column); 
    print(f"Counts for column '{column}':")
    stats = df[column].value_counts(sort=False);
    stats.sort_index()
    print(stats)
    print("\n")
    # break

print(df.info())
print(df.describe())


In [ ]:
 
all_keys =  df.keys().to_list()

all_keys.remove('id')
all_keys.remove(TARGET)
# result['Gender']
# print(all_keys, result)


for key in all_keys:

    try :
            # result = df.groupby([key,TARGET]).value_count()
            figure = plt.figure(figsize=(10,8))
            axes = figure.add_axes([0.1,0.2,0.9, 0.75])
             
            axes.set_ylabel('Percentage of total data surveyed')
            axes.set_xlabel(f"Analysing {key}, percentage of people with depression")

            grouped_counts = df.groupby([key, TARGET])[TARGET].value_counts()
            df2 = grouped_counts.reset_index(name='Count')
            # print(df2)
            uq_list = df2[key].unique()
            # print(uq_list)
            names = []
            vals = []
            data_list = []
            dep_list = []
            for name in uq_list : 
               total = df2.loc[df2[key] == name, 'Count'].sum()
               if total < 10:
                    continue
               depressed = df2.loc[(df2[key] == name ) & (df2['Depression'] == "1"), 'Count'].sum() 
               names.append(name) 
               dep_list.append(depressed)
            
               perc =math.floor(depressed*10000/total)/100
               vals.append(perc)    
               data_list.append([name,perc,depressed,total])
               print(f"{key} for {name:10} has  {perc:>6}% {depressed:>8}, {total:>8}")
            # axes.plot(names,vals,)
            axes.bar(names, vals,0.4)
            axes.set_xticks(names)
            axes.set_xticklabels(names, rotation=45, ha='right')
            # axes.set_yticklabels(vals);
            rot = 0
            if (len(vals)> 6):
                 rot = 45
            # for i in range(0,len(vals)):
            #     axes.text(i,vals[i],f"{dep_list[i]}",ha="center", rotation=rot )
            target = f'../reports/v1/figures/{slugify(key)}.png'
            figure.savefig(target)
            print(f"Generated {target} chart")

            stats = pd.DataFrame(np.array(data_list),columns=[key,'Percentage','Depressed','Total'])
             
            # print(stats)
            stats.to_csv(f"../reports/v1/{slugify(key)}.csv")
            # if (key == 'City'):
                #  break
    except Exception as err:
        print("Error while",err)
    # break


    # plt.bar(depression_counts['Age'], depression_counts['Count'], color='skyblue')
 